# **Compliance GPT with LangChain**

## **Setup**

In [1]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config

config = get_config()

In [3]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [4]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_ADA)

## **Load Vector Store Index**

In [5]:
from databases.vector_store import RedisIndexManager, PineconeIndexManager

# redis_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
# vector_store_ojk = redis_ojk.load_vector_index()

# redis_sikepo_ket = RedisIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config, db_id=0)
# vector_store_ket = redis_sikepo_ket.load_vector_index()

# redis_sikepo_rek = RedisIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config, db_id=0)
# vector_store_rek = redis_sikepo_rek.load_vector_index()

pinecone_ojk = PineconeIndexManager(index_name='ojk', embed_model=embed_model, config=config)
vector_store_ojk = pinecone_ojk.load_vector_index()

pinecone_ket = PineconeIndexManager(index_name='ketentuan-terkait', embed_model=embed_model, config=config)
vector_store_ket = pinecone_ket.load_vector_index()

pinecone_rek = PineconeIndexManager(index_name='rekam-jejak', embed_model=embed_model, config=config)
vector_store_rek = pinecone_rek.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
from databases.neo4j_graph_store import Neo4jGraphStore

neo4j_sikepo = Neo4jGraphStore(config=config)
graph = neo4j_sikepo.get_graph()

## **Get Retriever**

In [7]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo

retriever_ojk = get_retriever_ojk(vector_store=vector_store_ojk, top_n=5, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, llm_model=llm_model, embed_model=embed_model, config=config)

## **Create Chain**

In [8]:
from databases.chat_store import RedisChatStore, MongoDBChatStore

# chat_store = RedisChatStore(k=4, config=config, db_id=1)
chat_store = MongoDBChatStore(config=config, k=4)

In [9]:
from chain.rag_chain import create_chain_with_chat_history, create_chain
from retriever.retriever_sikepo.graph_cypher_retriever import graph_rag_chain


graph_chain = graph_rag_chain(llm_model, llm_model, graph=graph)

chain = create_chain(retriever_ojk, retriever_sikepo_rek, retriever_sikepo_ket, graph_chain, llm_model)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)

## **Invoke**

#### **Chat Store Monitor**

In [10]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Apa judul regulasi dengan nomor 66/POJK.04/2020?
AI: Pedoman Kontrak Investasi Kolektif Pemupukan Dana Tabungan Perumahan Rakyat

Source: [66/POJK.04/2020](https://www.ojk.go.id/id/regulasi/Documents/Pages/Pedoman-Kontrak-Investasi-Kolektif-Pemupukan--Dana-Tabungan-Perumahan-Rakyat/pojk%2066-2020.pdf)


In [12]:
# to make the chat history empty
chat_store.clear_all()

#### **With Question, Answer, and Context**

In [11]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Berapa SWDKLLJ dari buldozer?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

Error in RootListenersTracer.on_chain_end callback: ValueError()


{'question': 'Berapa SWDKLLJ dari buldozer?',
 'answer': {'rewrited question': 'Berapa SWDKLLJ dari buldozer?',
  'answer': 'Saya tidak tahu mengenai hal tersebut.',
  'context': [Document(metadata={'Jenis Ketentuan': 'SEBI', 'Judul Ketentuan': 'Batas Maksimum Penyaluran Dana Bank Pembiayaan Rakyat Syariah', 'Ketentuan': 'Romawi III Sub 6', 'Kodifikasi Ketentuan': 'BPR Syariah', 'Nomor Ketentuan': '13/17/DPbS', 'Referensi': '-', 'Tanggal Ketentuan': '30 Mei 2011', 'relevance_score': 0.0002824081}, page_content='- Salam untuk pembelian beras jenis IR45 sebanyak 2 (dua) ton kepada nasabah D sebesar Rp50.000.000,00 (lima puluh juta rupiah), jangka waktu 6 (enam) bulan, tanggal akad 15 Maret 2011.\n- Ijarah atas hak penggunaan kios yang diperoleh dari Tuan F dengan harga perolehan sewa sebesar Rp120.000.000,00 (seratus dua puluh juta rupiah) selama 2 (dua) tahun kepada nasabah E dan BPRS menetapkan pendapatan sewa (ujroh) sebesar Rp20.000.000,00 (dua puluh juta rupiah), jangka waktu 2 (dua

In [17]:
print(response['context'])

[
  {
    "metadata": {
      "doc_id": 159.0,
      "effective_date": "14 Juni 2023",
      "file_url": "https://www.ojk.go.id/id/regulasi/Documents/Pages/POJK-APU-PPT-dan-PPPSPM-di-SJK/POJK%208%20TAHUN%202023%20-%20PENERAPAN%20PROGRAM%20APU%20PPT%20DAN%20PPSPM%20DI%20SJK.pdf",
      "page_number": 113.0,
      "regulation_number": "8 Tahun 2023",
      "regulation_type": "Peraturan OJK",
      "sector": "Perbankan,  Pasar Modal,  IKNB,  Syariah,  EPK",
      "subsector": "Peraturan Lainnya",
      "title": "Penerapan Program Anti Pencucian Uang, Pencegahan Pendanaan Terorisme, dan Pencegahan Pendanaan Proliferasi Senjata Pemusnah Massal di Sektor Jasa Keuangan",
      "relevance_score": 0.071073666
    },
    "page_content": "kenakan selama tahun 2024 atas pelanggaran yang \ndilakukan PJK tersebut di luar pelanggaran pelaporan \nmenggunakan \nperhitungan \n1% \n(satu \npersen) \ndikali \nRp1.000.000.000,00 (satu miliar rupiah), yaitu sebesar \nRp10.000.000,00 (sepuluh juta rupiah). \

#### **Streaming**

In [19]:
# from chain.rag_chain import print_answer_stream

# print_answer_stream(chain=chain_history, question="Siapa itu lionel messi?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Saya minta maaf, tetapi pertanyaan yang Anda ajukan tidak terkait dengan konteks dan informasi metadata yang diberikan. Silakan ajukan pertanyaan lain yang terkait dengan kepatuhan perbankan di Indonesia.